# Step Response Comparison on Go1 (Physical vs Mujoco)

In [114]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path
import numpy as np
import pickle
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
from bokeh.palettes import Category20_12
from bokeh.models import Span, LinearAxis, Range1d, HoverTool
from bokeh.layouts import column, row, widgetbox

In [2]:
output_notebook()

Loading BokehJS ...

In [12]:
JOINT_NAMES = []
for prefix in ["FR", "FL", "RR", "RL"]:
    for postfix in ["hip", "thigh", "calf"]:
        JOINT_NAMES.append(f"{prefix}_{postfix}")

In [104]:
class Comparator(object):
    def __init__(self, mjc_data_path: Path, phy_data_path: Path):
        with open(mjc_data_path, "rb") as f:
            self._mjc = pickle.load(f)
        with open(phy_data_path, "rb") as f:
            self._phy = pickle.load(f)
        
        # Use to set the index to start the visualization. This can be used to tune the time alignment.
        self._start_mjc = 0
        self._start_phy = 0
        
    def align_time(self, start_mjc: int, start_phy: int):
        self._start_mjc = start_mjc
        self._start_phy = start_phy
        
    def compare_joint(self, joint_id: int, plot_width: int = 1024):
        j = joint_id
        t_mjc = self._start_mjc
        t_phy = self._start_phy
        
        fig = figure(title=f"Joint {joint_id}", x_axis_label="time", 
                     y_axis_label="rad", y_range=Range1d(start=-4, end=4), plot_width=plot_width)
        fig.extra_y_ranges = {"torque": Range1d(start=-40, end=40)}
        fig.add_layout(LinearAxis(y_range_name="torque", axis_label='torque'), 'right')
        
        num_items = 0
        def _plot(data: np.ndarray, name: str, color_id: int, phy: bool = False):
            nonlocal num_items
            kwargs = {
                "pos": {"line_width": 2},
                "vel": {"line_width": 2, "line_dash": "dotted"},
                "tau": {"line_width": 1, "y_range_name": "torque"},
                "cmd": {"line_width": 1, "line_dash": "dashed"},
                
            }[name]
            postfix = "phy" if phy else "sim"
            fig.line(np.arange(data.shape[0]), data,
                     legend_label=f"{JOINT_NAMES[j]}_{name}@{postfix}",
                     color=Category20_12[color_id], **kwargs)
            num_items += 1
        
        _plot(self._mjc["qpos"][t_mjc:, j], "pos", 0)
        _plot(self._mjc["qvel"][t_mjc:, j], "vel", 0)
        _plot(self._mjc["torque"][t_mjc:, j], "tau", 1)
        _plot(self._mjc["cmd"][t_mjc:, j], "cmd", 2)
        
        _plot(self._phy["qpos"][t_phy:, j], "pos", 5, phy=True)
        _plot(self._phy["qvel"][t_phy:, j], "vel", 5, phy=True)
        _plot(self._phy["torque"][t_phy:, j], "tau", 6, phy=True)
        _plot(self._phy["cmd"][t_phy:, j], "cmd", 7, phy=True)
        
        hover = HoverTool(tooltips=[("x", "@x"), ("y1", "@y")], renderers=[fig.renderers[k] for k in range(num_items)])
        fig.add_tools(hover)
        
        fig.legend.location = "top_right"
        fig.legend.click_policy = "hide"
        return fig
    
    def compare_gyro(self, plot_width: int = 1024):
        t_mjc = self._start_mjc
        t_phy = self._start_phy
        
        fig = figure(title="Gyro", x_axis_label="time", y_axis_label="rad/s", plot_width=plot_width)
        
        def _plot(data: np.ndarray, id: int, phy: bool = False):
            kwargs = {}
            if phy:
                kwargs = {"line_dash": "dashed"}
            postfix = "phy" if phy else "sim"
            fig.line(np.arange(data.shape[0]), data,
                     legend_label=f"gyro{id}@{postfix}",
                     color=Category20_12[id + (3 if phy else 0)], line_width=2, **kwargs)
            
        _plot(self._mjc["gyro"][t_mjc:, 0], 0)
        _plot(self._mjc["gyro"][t_mjc:, 1], 1)
        _plot(self._mjc["gyro"][t_mjc:, 2], 2)
        
        _plot(self._phy["gyro"][t_phy:, 0], 0, phy=True)
        _plot(self._phy["gyro"][t_phy:, 1], 1, phy=True)
        _plot(self._phy["gyro"][t_phy:, 2], 2, phy=True)
      
        hover = HoverTool(tooltips=[("x", "@x"), ("y1", "@y")], renderers=[fig.renderers[k] for k in range(6)])
        fig.add_tools(hover)

        fig.legend.location = "top_right"
        fig.legend.click_policy = "hide"
        return fig

In [105]:
comparator = Comparator(
    mjc_data_path=Path("/home/breakds/syncthing/workspace/hobot/calibrate_go1/mujoco_step_response.pkl"),
    phy_data_path=Path("/home/breakds/syncthing/workspace/hobot/calibrate_go1/physical_step_response.pkl"))

In [ ]:
layout = column(
    comparator.compare_joint(1),
    comparator.compare_joint(2),
    comparator.compare_joint(4),
    comparator.compare_joint(5),
    comparator.compare_joint(7),
    comparator.compare_joint(8),
    comparator.compare_joint(10),
    comparator.compare_joint(11),
    comparator.compare_gyro())
show(layout)

In [115]:
output_file("/home/breakds/syncthing/workspace/hobot/calibrate_go1/baseline.html", title="MuJoCo VS Physical")
save(layout)

'/home/breakds/syncthing/workspace/hobot/calibrate_go1/baseline.html'